In [ ]:
import torch as th

import matplotlib.pyplot as plt
import time

import sys
import os
from __future__ import division
import os
import urllib, cStringIO
import pymongo as pm ## first establish ssh tunnel to server where database is running
import base64
import numpy as np
from numpy import *
import PIL
from PIL import Image
import base64
import matplotlib
%matplotlib inline
from matplotlib import pylab, mlab, pyplot
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from matplotlib.path import Path
import matplotlib.patches as patches
import cv2
import pandas as pd
from svgpathtools import parse_path
import svg_distance_helpers as rsh
import torch
from torch.autograd import Variable
import SimpleITK as sitk
from torch.autograd import Variable


#### File Hierarchy

In [ ]:
# directory & file hierarchy
analysis_dir = os.getcwd()
data_dir = os.path.join(analysis_dir,"photodraw_tracing")
ref_dir = os.path.join(analysis_dir, 'tracing_ref_400')
tracing = ['square','shape', 'this circle']

#### Set Up

In [ ]:
# generate reference shapes
really_run_this = 1
if really_run_this:
    ref_path = []
    for t in tracing:
        print t
        fname = os.path.join(ref_dir, '{}.png'.format(t))
        img_ref = cv2.imread(fname)
        img_ref = rsh.color_to_num(img_ref, threshold=0, white_background=False)
        ofname = os.path.join(ref_dir, '{}_ref.png'.format(t))
        cv2.imwrite(ofname, 255-img_ref*255)
        ref_path.append(ofname)
    

In [ ]:
# Overlap the drawing on reference images
iteration_name = ['CDM_photodraw_e1', 'cdm_run_v3', 'cdm_run_v4']
dir_name = ['photodraw_e1', 'museumstation_v3', 'museumstation_v4']
tracing = ['square', 'shape', 'this circle']
overlap_dir = os.path.join(analysis_dir, 'tracing_overlap')
if not os.path.exists(overlap_dir):
    os.makedirs(overlap_dir)

for di, dname in enumerate(dir_name):
    iter_dir = os.path.join(overlap_dir, dname)
    if not os.path.exists(iter_dir):
        os.makedirs(iter_dir)
        
    current_data_dir = os.path.join(analysis_dir,"tracing_{}".format(iteration_name[di]))

    # get tracings from the data folder
    for ti, t in enumerate(tracing):
            
        if dname == 'photodraw_e1' and (t == 'square' or t == 'shape'):
            t = 'this ' + t
        t_dir = os.path.join(current_data_dir, t)
        overlap_t = os.path.join(iter_dir, t)
        
        if not os.path.exists(overlap_t):
                os.makedirs(overlap_t)
                
        if os.path.exists(t_dir):
            for fname in os.listdir(t_dir):
                if fname.startswith('age'):
                    img_draw = os.path.join(current_data_dir, t_dir, fname)
                    img_ref = ref_path[ti]

                    draw = cv2.imread(img_draw).astype(int16)
                    ref = cv2.imread(img_ref).astype(int16)
                    img = ref
                    img[np.sum(ref, axis=2)<255*3] = [150, 150, 150]
                    img[np.sum(draw, axis=2)<255*3] = [40, 40, 170]


                    cv2.imwrite(os.path.join(iter_dir,t, fname), img)

#### upload stims to s3

In [ ]:
import boto
runThis = 0
if runThis:
    conn = boto.connect_s3()
    b = conn.create_bucket(bucket_name) ### if bucket already exists, then get_bucket, else create_bucket
    for ind,this_triplet in enumerate(triplet_dirs):
        ims = [i for i in os.listdir(this_triplet) if i[-3:]=='png']
        for im in ims:
            print ind, im
            k = b.new_key(im)
            k.set_contents_from_filename(os.path.join(this_triplet,im))
            k.set_acl('public-read')

#### build stimulus dictionary & upload metadata to mongo

In [ ]:
# read in data from csv files
cdm_v3 = pd.read_csv('MuseumStation_Tracing_Descriptives_cdm_run_v3.csv')
cdm_v4 = pd.read_csv('MuseumStation_Tracing_Descriptives_cdm_run_v4.csv')
photodraw = pd.read_csv('Photodraw_Tracing_Descriptives_CDM_photodraw_e1.csv')
limit = 200
overall = photodraw

for t in tracing:
    for a in range(2,11):
        cdm_v3_tage = cdm_v3.loc[(cdm_v3['age'] == 'age{}'.format(a)) & (cdm_v3['category']== t)]
        v3_size = cdm_v3_tage.shape[0]                 
        if v3_size > limit:
            overall = pd.concat([overall, cdm_v3_tage[:200]])
            
        else:
            overall = pd.concat([overall, cdm_v3_tage]) 
            
            cdm_v4_tage = cdm_v4.loc[(cdm_v4['age'] == 'age{}'.format(a)) & (cdm_v4['category']== t)]
            
            overall = pd.concat([overall, cdm_v4_tage[:limit-v3_size]])

print overall.shape
for t in tracing:
    for a in range(2,11):
        print t, a, overall.loc[(overall['age'] == 'age{}'.format(a)) & ( (overall['category'] == t) | (overall['category'] == 'this {}'.format(t)) )].shape

overall = overall.drop(['filename', "Unnamed: 0"], axis=1)
overall.reset_index(inplace = True)

In [ ]:
overall['exp_name'] = 0
overall['iteration_name'] = 0
overall['img_url'] = 0
overall['number_rating_levels'] = 5
overall['dynamic_rating'] = False
overall['lower_bound'] = 'NaN'
overall['upper_bound'] = 'NaN' 

for index, row in overall.iterrows():
    sid = row['session_id']
    if sid.startswith('CDM'):
        overall.loc[index,'exp_name'] = 'photodraw'
        overall.loc[index,'iteration_name'] = 'e1'
    else:
        overall.loc[index,'exp_name'] = 'museumstation'
        overall.loc[index,'iteration_name'] = sid.split('_')[2][:2]
    
    fname = '{}_{}_{}.png'.format(row['age'], row['session_id'], row['category'])
    exp_folder = overall.loc[index,'exp_name'] + '_' + overall.loc[index,'iteration_name']
    overall.loc[index,'img_url'] = 'https://s3.amazonaws.com/tracing_overlap/{}/{}/{}'.format(exp_folder, row['category'], fname)

overall = overall.drop('index', axis=1)
overall.to_csv('tracing_eval_dict.csv')

In [ ]:
## convert csv file to json file
import json
overall = pd.read_csv('tracing_eval_dict.csv')
stimdict = overall.to_dict(orient='records') 
with open('{}.js'.format('tracing_eval_dict'), 'w') as fout:
    json.dump(stimdict, fout)

In [ ]:
### next todo is to upload this JSON to initialize the new stimulus collection
print('next todo is to upload this JSON to initialize the new stimulus collection...')
import json
J = json.loads(open('{}.js'.format(dataset_name),mode='ru').read())

In [ ]:
##assert len(J)==len(all_files)
print 'dataset_name: {}'.format(dataset_name)
print len(J)

In [ ]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']
coll = db[dataset_name]

In [ ]:
## actually add data now to the database
reallyRun = 1
if reallyRun:
    for (i,j) in enumerate(J):
        if i%100==0:
            print ('%d of %d' % (i,len(J)))
        coll.insert_one(j)

In [ ]:
## check how many records have been retrieved
a = coll.find({'shuffler_ind':{'$gte':0}})
numGames = []
for rec in a:
    numGames.append(len(rec['games']))
b = np.array(numGames)
print np.mean(b>0)